Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.


In [2]:
data1 = pd.read_csv('spam.csv', encoding = 'latin-1') # Kaggle dataset
data1 = data1.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
data1.head()

data2 = pd.ExcelFile('revisedindiandataset.xls')
data2 = data2.parse(0)
data2 = data2.drop(['code'], axis = 1)
data2.head()

data = pd.concat([data1, data2])

# Dividing sentences for word2vec training

In [3]:
sentences = []
pro_data = []
for row in data.itertuples():
    rev = str(row[2])
    words = text_to_word_sequence(rev)
    
    if(len(words) == 0):
        pro_data.append(words)
        continue
    
    if(len(words) == 1):
        te = ['</s>', words[0], '</e>']
        sentences.append(te)
        pro_data.append(words)
        continue
    
    te = ['</s>', words[0], words[1]]
    sentences.append(te)
    
    for i in range(1,len(words) - 1):
        te = [words[i-1], words[i], words[i+1]]
        sentences.append(te)
        
    te = [words[len(words) - 2], words[len(words) - 1], '</e>']
    sentences.append(te)   
    
    pro_data.append(words)

In [4]:
print(data['msg'].iloc[0])
print(sentences[0])
print(sentences[1])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
['</s>', 'go', 'until']
['go', 'until', 'jurong']


# word2vec training

In [5]:
size = 150
from gensim.models import Word2Vec
model = Word2Vec(sentences, min_count=1, size = size)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
model['god']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ -7.63274357e-02,   6.56855106e-02,  -1.81274548e-01,
        -5.93337454e-02,  -9.38541666e-02,   1.24068230e-01,
        -7.51162097e-02,  -8.01607892e-02,  -2.70406187e-01,
        -4.93438654e-02,   1.63160726e-01,  -1.68489933e-01,
         3.59951496e-01,  -1.22219384e-01,   6.20605871e-02,
         7.76917562e-02,   1.03679672e-01,  -6.13218658e-02,
        -6.47999525e-01,  -8.04246217e-02,   5.40767275e-02,
        -2.24839034e-03,   1.24966413e-01,  -2.33261794e-01,
        -2.09508583e-01,  -4.87217680e-02,   3.39099765e-02,
        -1.16569303e-01,   8.83139595e-02,   1.72589257e-01,
         1.43201202e-01,  -3.36733386e-02,  -3.65715623e-01,
        -1.03109099e-01,   3.59832495e-03,   9.67013463e-02,
         3.12370714e-02,   2.34864582e-03,  -1.18923955e-01,
        -4.34898883e-01,   2.91270882e-01,   2.56589085e-01,
        -9.66653377e-02,  -4.09884453e-01,  -1.49413124e-01,
        -1.47451684e-02,   1.25115275e-01,  -9.65339243e-02,
        -4.38281859e-04,

# Pre - padding

In [7]:
data_vec = []
msg_limit = 200
for i in range(len(pro_data)):
    empty = [0 for i in range(size)]
    temp = [[0 for i in range(size)] for j in range(msg_limit)]
    for j in range(len(pro_data[i])-1, -1, -1):
        temp[199 - (len(pro_data[i])-1-j)] = model[pro_data[i][j]]
    data_vec.append(temp)
data_vec = np.array(data_vec)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


splitting data for training and testing purposes

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    data_vec, data['label'], test_size=0.30, random_state=0)

Y_train = pd.get_dummies(Y_train).values

# Model

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout

classifier = Sequential()
classifier.add(LSTM(200, input_shape =(200, size)))
classifier.add(Dropout(0.2))
classifier.add(Dense(2, activation='softmax'))
classifier.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               280800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 281,202
Trainable params: 281,202
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
classifier.fit(X_train, Y_train, epochs = 7, batch_size = 50) # training the model

Epoch 1/7
7097/7097 [==============================] - 64s 9ms/step - loss: 0.2242 - acc: 0.9139
Epoch 2/7
7097/7097 [==============================] - 40s 6ms/step - loss: 0.1351 - acc: 0.9515
Epoch 3/7
7097/7097 [==============================] - 39s 6ms/step - loss: 0.1064 - acc: 0.9593
Epoch 4/7
7097/7097 [==============================] - 39s 6ms/step - loss: 0.0947 - acc: 0.9672
Epoch 5/7
7097/7097 [==============================] - 40s 6ms/step - loss: 0.0884 - acc: 0.9684
Epoch 6/7
7097/7097 [==============================] - 43s 6ms/step - loss: 0.0708 - acc: 0.9751
Epoch 7/7
7097/7097 [==============================] - 42s 6ms/step - loss: 0.0746 - acc: 0.9724


# Evaluating Accuracy

In [11]:
Y_pr = classifier.predict(X_test)

Y_pred = []

for i in range(len(Y_pr)):
    if Y_pr[i][0] >= Y_pr[i][1]:
        Y_pred.append('ham')
    else:
        Y_pred.append('spam')

In [12]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)  
print(cm)

[[2398   40]
 [  61  543]]
